# np.matmul (\__matmul\__, @)

https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html

The behavior depends on the arguments in the following way.

* If both arguments are 2-D they are multiplied like conventional matrices.
* **If either argument is N-D, N > 2, it is treated as a stack of matrices residing in the last two indexes and broadcast accordingly.**
* If the first argument is 1-D, it is promoted to a matrix by prepending a 1 to its dimensions. After matrix multiplication the prepended 1 is removed.
* If the second argument is 1-D, it is promoted to a matrix by appending a 1 to its dimensions. After matrix multiplication the appended 1 is removed.

In [1]:
import numpy as np

A1 = np.random.rand(2, 2)
A2 = np.random.rand(2, 2)
A3 = np.random.rand(2, 2)

B1 = np.random.rand(2, 2)
B2 = np.random.rand(2, 2)
B3 = np.random.rand(2, 2)

A1.shape

(2, 2)

In [2]:
A = np.array([A1, A2, A3])  # or: np.stack
B = np.array([B1, B2, B3])

A.shape

(3, 2, 2)

In [3]:
# multiply one-by-one
C1 = A1 @ B1
C2 = A2 @ B2
C3 = A3 @ B3

C1

array([[0.74582686, 0.79862583],
       [0.19346337, 0.23241742]])

In [4]:
C = np.array([C1, C2, C3])
C

array([[[0.74582686, 0.79862583],
        [0.19346337, 0.23241742]],

       [[0.85269621, 0.43787267],
        [0.06713207, 0.12352583]],

       [[0.2885247 , 0.27686453],
        [0.60433743, 0.21696731]]])

In [5]:
# multiply the whole stack using np.matmul
D = A @ B  # or: np.matmul(A, B)
D

array([[[0.74582686, 0.79862583],
        [0.19346337, 0.23241742]],

       [[0.85269621, 0.43787267],
        [0.06713207, 0.12352583]],

       [[0.2885247 , 0.27686453],
        [0.60433743, 0.21696731]]])

In [6]:
np.array_equal(C, D)

True

In [7]:
C - D

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

## $ \Sigma^f = J\,\Sigma\,J^\mathbf{T} $

In [8]:
J = np.random.rand(100, 2, 2)
J[0]

array([[0.05830188, 0.41095085],
       [0.26963458, 0.32457398]])

In [9]:
Sigma = np.random.rand(100, 2, 2)
Sigma[42]

array([[0.09609754, 0.02023986],
       [0.55520642, 0.17045759]])

In [10]:
J.T.shape  # .T transposes the first axis, not the "second ones"

(2, 2, 100)

In [11]:
J_T = J.transpose(0, 2, 1)  # pass new order of axes
J_T

array([[[0.05830188, 0.26963458],
        [0.41095085, 0.32457398]],

       [[0.55513097, 0.04873309],
        [0.42873459, 0.34469663]],

       [[0.52070871, 0.10117999],
        [0.59854587, 0.85667173]],

       [[0.36143309, 0.18185472],
        [0.76001491, 0.11456041]],

       [[0.32926079, 0.56943181],
        [0.64480997, 0.17262404]],

       [[0.01434959, 0.87611692],
        [0.72015236, 0.2240025 ]],

       [[0.18956353, 0.56467238],
        [0.21767799, 0.98324272]],

       [[0.6447021 , 0.81384778],
        [0.27004912, 0.31684358]],

       [[0.39548703, 0.56390297],
        [0.57427952, 0.37403219]],

       [[0.72084228, 0.15656151],
        [0.04650913, 0.04638091]],

       [[0.31972654, 0.32557627],
        [0.7077927 , 0.04540916]],

       [[0.188007  , 0.99469782],
        [0.93696269, 0.29295202]],

       [[0.016021  , 0.56848346],
        [0.60472948, 0.60643295]],

       [[0.70135849, 0.98617818],
        [0.53511614, 0.43346536]],

       [[0.01779983,

In [12]:
J_T.shape == (100, 2, 2)

True

In [13]:
np.array_equal(
    J[42].T,
    J_T[42]
)

True

In [14]:
Sigma_f = J @ Sigma @ J_T

Sigma_f.shape

(100, 2, 2)

In [15]:
np.array_equal(
    Sigma_f[42],
    J[42] @ Sigma[42] @ J[42].T
)

True

## Speed and heading to velocity_otg uncertainty propagation

$
\begin{aligned}
    V_\text{x} &= S \cos{H} \\[0.1cm]
    V_\text{y} &= S \sin{H}
\end{aligned}
$

$
\mathrm{J}_f = \begin{bmatrix}
        \dfrac{\partial V_\text{x}}{\partial S} & \dfrac{\partial V_\text{x}}{\partial H} \\[0.4cm]
        \dfrac{\partial V_\text{y}}{\partial S} & \dfrac{\partial V_\text{y}}{\partial H}
    \end{bmatrix} = \begin{bmatrix}
        \cos{H} & -S\sin{H}  \\[0.2cm]
        \sin{H} & S\cos{H}
    \end{bmatrix}
$

### Generate test input signals

In [16]:
N = 123456  # number of rows in signals dataframe

speed = np.random.rand(N)  # from motion_model_state_suppl[2]
heading = np.random.rand(N)  # from motion_model_state_suppl[4]
speed_variance = np.random.rand(N)  # from motion_model_variances_suppl[2]
heading_variance = np.random.rand(N)  # from motion_model_variances_suppl[4]
speed_heading_covariance = np.random.rand(N)  # from motion_model_covariances_suppl[1]

speed.shape, speed

((123456,),
 array([0.61692749, 0.99912519, 0.7916012 , ..., 0.60839912, 0.51307943,
        0.49421849]))

### Calculate Jacobi matrix values

In [17]:
j11 = np.cos(heading)
j12 = -speed * np.sin(heading)  # may be optimized: j12 = -speed * j21
j21 = np.sin(heading)
j22 = speed * np.cos(heading)  # may be optimized: j22 = speed * j11

j11.shape

(123456,)

### Method 1. Using matrix multiplication for each row

In [18]:
%%time
# this may take a long time... (about 2-3 seconds)

for i in range(N):
    J_i = np.array([
        [ np.cos(heading[i]), -speed[i] * np.sin(heading[i]) ],
        [ np.sin(heading[i]),  speed[i] * np.cos(heading[i]) ],
    ])
    Sigma_i = np.array([
        [ speed_variance[i], speed_heading_covariance[i] ],
        [ speed_heading_covariance[i], heading_variance[i] ],
    ])
    
    J_i @ Sigma_i @ J_i.T

Wall time: 3.5 s


![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSVgUbxPl3uTPRGXJkKqvyHzImBnPmmqU9KC87XDREztFuPetu1)

Extracting values from output matrix and assigning them to respsective dataframe cells may  eve double the calculation time!

### Method 2. Using matmul on stacked arrays

In [19]:
J = np.array([[j11, j12], [j21, j22]])
J.shape  # :(

(2, 2, 123456)

In [20]:
J = np.array([[j11, j12], [j21, j22]]).transpose(2, 0, 1)  # all magic happens here
J.shape

(123456, 2, 2)

In [21]:
J[42]

array([[ 0.95400471, -0.12796216],
       [ 0.29979162,  0.4072045 ]])

In [22]:
np.array([
    [j11[42], j12[42]],
    [j21[42], j22[42]]
])
# is this same as J[42]?

array([[ 0.95400471, -0.12796216],
       [ 0.29979162,  0.4072045 ]])

In [23]:
np.array_equal(
    J[42],
    np.array([
        [j11[42], j12[42]],
        [j21[42], j22[42]]
    ])
)

True

In [24]:
Sigma = np.array([
    [speed_variance, speed_heading_covariance],
    [speed_heading_covariance, heading_variance],
]).transpose(2, 0, 1)

Sigma.shape

(123456, 2, 2)

In [25]:
J_T = J.transpose(0, 2, 1)  # axis 0 stays the same, swap axes 1 and 2
J.shape

(123456, 2, 2)

In [26]:
np.array_equal(
    J_T[42],
    J[42].T
)

True

In [27]:
%%time

Sigma_f = J @ Sigma @ J_T

Sigma_f.shape

Wall time: 13 ms


(123456, 2, 2)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT-N7-PgatzxayzpHxJOrxtsUic22KQSOqKT0ZQgJ4LcyCeAV0&s)

### Comparision of results

In [28]:
J_42 = np.array([
    [ np.cos(heading[42]), -speed[42] * np.sin(heading[42]) ],
    [ np.sin(heading[42]),  speed[42] * np.cos(heading[42]) ],
])
Sigma_42 = np.array([
    [ speed_variance[42], speed_heading_covariance[42] ],
    [ speed_heading_covariance[42], heading_variance[42] ],
])

Sigma_f_42 = J_42 @ Sigma_42 @ J_42.T
Sigma_f_42

array([[0.01020399, 0.10011967],
       [0.10011967, 0.12065377]])

In [29]:
Sigma_f[42]

array([[0.01020399, 0.10011967],
       [0.10011967, 0.12065377]])

In [30]:
np.array_equal(
    Sigma_f_42,
    Sigma_f[42]
)

False

In [31]:
Sigma_f_42 - Sigma_f[42]

array([[-1.73472348e-18,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00]])

In [32]:
np.allclose(
    Sigma_f_42,
    Sigma_f[42]
)

True

### Assigning to signals dataframe columns

In [33]:
Sigma_f.shape

(123456, 2, 2)

In [34]:
velocity_otg_variance_x = Sigma_f[:,0,0]  # assign to signals['velocity_otg_variance_x']
velocity_otg_variance_y = Sigma_f[:,1,1]  # assign to signals['velocity_otg_variance_y']
velocity_otg_covariance = Sigma_f[:,0,1]  # assign to signals['velocity_otg_covariance']

velocity_otg_variance_x.shape, velocity_otg_variance_x

((123456,),
 array([ 0.56131412,  0.360243  ,  0.12132119, ..., -0.00803077,
         0.68500845,  0.01794193]))

![](https://upload.wikimedia.org/wikipedia/commons/e/ea/Thats_all_folks.svg)